In [51]:
!pip install openpyxl

  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


You should consider upgrading via the 'c:\users\ekaap\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [39]:
import pandas as pd
from collections import Counter
import openpyxl
import numpy as np

In [57]:
df = pd.read_json(open("soal-2.json", encoding='utf-8'), orient='records')

In [58]:
# remove non alphabet
df['komoditas_clean'] = df['komoditas'].str.lower().str.replace('[^a-zA-Z0-9]', ' ')

<ipython-input-58-1e36a59bdeb0>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['komoditas_clean'] = df['komoditas'].str.lower().str.replace('[^a-zA-Z0-9]', ' ')


In [28]:
result = Counter(" ".join(df['komoditas_clean'].values.tolist()).split(" ")).items()
result

dict_items([('lele', 642), ('', 1329), ('bawal', 124), ('nila', 320), ('kakap', 127), ('kembung', 147), ('tongkol', 289), ('salem', 19), ('ikan', 214), ('salam', 1), ('kerapu', 25), ('mas', 417), ('pindang', 2), ('gurame', 83), ('patin', 48), ('bandeng', 15), ('jaer', 4), ('mujaer', 86), ('mujair', 49), ('peda', 1), ('kepala', 2), ('dan', 23), ('gurami', 3), ('cumi', 11), ('lelw', 1), ('emas', 1), ('kembug', 1), ('udang', 12), ('kepiting', 5), ('kerang', 4), ('gabus', 1), ('parin', 1), ('ayam', 3), ('laut', 24), ('tawar', 4), ('soto', 1), ('babat', 2), ('kikil', 1), ('sea', 1), ('food', 1), ('barunang', 1), ('pecel', 5), ('bebek', 2), ('goreng', 4), ('jambal', 1), ('gembung', 1), ('dll', 4), ('bakar', 1), ('sate', 1), ('usus', 1), ('nasi', 2), ('uduk', 2), ('sotobayam', 1), ('munkar', 2), ('cakalang', 2), ('majaer', 1), ('mujir', 1), ('krapu', 1), ('kerpu', 1), ('man', 1), ('tingkol', 1), ('pari', 1), ('baracuda', 1), ('merah', 11), ('ikn', 1), ('impun', 1), ('muajir', 2), ('kele', 1),

In [59]:
# Replace values step 1
replacement = {
    "salam": "salem",
    "mujaer": "mujair",
    "majaer": "mujair",
    "mujir": "mujair",
    "muajir": "mujair",
    "ikan": "",
    "ikn": "",
    r"\bdan\b": "",
    "gurami": "gurame",
    "lelw": "lele",
    "ikanlele": "lele",
    "kele": "lele",
    "emas": "mas",
    "kembug": "kembung",
    "parin": "patin",
    "gembung": "kembung",
    "krapu": "kerapu",
    "kerpu": "kerapu",
    "tingkol": "tongkol",
    "tngkol": "tongkol",
    "nilem": "nila",
    r"\bnil\b": "",
    r"\bkepala\b": "",
    r"\bmunkar\b": "mujair",
    r"\bmas\b": "mas",
    r"\bpari\b": "laut",
    r"\bbaracuda\b": "laut",
    r"\betong\b": "laut",
    r"\btenggiri\b": "laut",
    r"\bmerah\b": "",
    r"\bhitam\b": ""
}

df['komoditas_clean'] = df['komoditas_clean'].replace(replacement, regex=True)

In [30]:
result = Counter(" ".join(df['komoditas_clean'].values.tolist()).split(" ")).items()
result

dict_items([('lele', 645), ('', 1582), ('bawal', 124), ('nila', 321), ('kakap', 127), ('kembung', 149), ('tongkol', 291), ('salem', 20), ('kerapu', 27), ('mas', 418), ('pindang', 2), ('gurame', 86), ('patin', 49), ('bandeng', 15), ('jaer', 4), ('mujair', 141), ('peda', 1), ('cumi', 11), ('udang', 12), ('kepiting', 5), ('kerang', 4), ('gabus', 1), ('ayam', 3), ('laut', 28), ('tawar', 4), ('soto', 1), ('babat', 2), ('kikil', 1), ('sea', 1), ('food', 1), ('barunang', 1), ('pecel', 5), ('bebek', 2), ('goreng', 4), ('jambal', 1), ('dll', 4), ('bakar', 1), ('sate', 1), ('usus', 1), ('nasi', 2), ('uduk', 2), ('sotobayam', 1), ('cakalang', 2), ('man', 1), ('impun', 1)])

In [60]:
# Mark possibly irrelevant entries
keywords = [
    "ayam", "soto", "babat", "kikil", "nasi", "uduk", "sotobayam", "sea", "food", "pecel", "bebek",
    "goreng", "dll", "bakar", "sate", "usus"
]

df["possibly_irr"] = df['komoditas_clean'].apply(lambda x: 1 if any(i in x for i in keywords) else 0)

# Remove possibly irrelevant entries
df = df[df["possibly_irr"] == 0]

In [25]:
df.to_excel("df.xlsx")

In [61]:
# Remove berat without number

df= df[df["berat"].str.contains(r'[0-9]')]

# Replace comma with space

df['berat_clean'] = df['berat'].str.lower().str.replace(',', ' ')

In [63]:
# Mark one word only

df['one_word'] = df['komoditas_clean'].str.match('^[a-zA-Z0-9_]+$')

In [102]:
# Process one word
df_one_word = df[df['one_word'] == True].reset_index()

df_one_word["berat_final_1"] = df_one_word["berat_clean"].str.extract(pat= r'([0-9](?=\skg))')
df_one_word["berat_final_2"] = df_one_word["berat_clean"].str.extract(pat= r'([0-9](?=kg))')
df_one_word["berat_final_3"] = df_one_word["berat_clean"].str.extract(pat= r'([0-9])')

df_one_word["berat_final"] = None

for i in range(len(df_one_word["berat_final"])):
    if str(df_one_word["berat_final_1"][i]) != 'nan':
        df_one_word["berat_final"][i] = df_one_word["berat_final_1"][i]
    elif str(df_one_word["berat_final_2"][i]) != 'nan':
        df_one_word["berat_final"][i] = df_one_word["berat_final_2"][i]
    else:
        df_one_word["berat_final"][i] = df_one_word["berat_final_3"][i]

df_one_word = df_one_word[['komoditas_clean', 'berat_final']]
df_one_word.columns = ['komoditas', 'berat']

<ipython-input-102-31979056b6b6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_word["berat_final"][i] = df_one_word["berat_final_2"][i]
<ipython-input-102-31979056b6b6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_word["berat_final"][i] = df_one_word["berat_final_1"][i]
<ipython-input-102-31979056b6b6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_word["berat_final"][i] = df_one_word["berat_final_3"][i

In [105]:
counts = df_one_word.groupby(['komoditas'])['berat'].count().sort_values(ascending=False)